<a href="https://colab.research.google.com/github/oneir0mancer/stable-diffusion-diffusers-colab-ui/blob/main/SDXL_diffusers_ui.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://huggingface.co/docs/diffusers/v0.19.3/api/pipelines/stable_diffusion/stable_diffusion_xl

In [ ]:
#@title #Install dependencies
!git clone https://github.com/oneir0mancer/stable-diffusion-diffusers-colab-ui.git StableDiffusionUi
!pip install -r StableDiffusionUi/requirements.txt

!mkdir -p outputs/{txt2img,img2img}

In [ ]:
import torch
from diffusers import DiffusionPipeline, AutoencoderKL

In [ ]:
vae = AutoencoderKL.from_pretrained("madebyollin/sdxl-vae-fp16-fix",
                                    torch_dtype=torch.float16)

In [ ]:
base = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    vae=vae,
    torch_dtype=torch.float16,
    variant="fp16",
    use_safetensors=True,
).to("cuda")
base.enable_xformers_memory_efficient_attention()

refiner = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-refiner-1.0",
    text_encoder_2=base.text_encoder_2,
    vae=base.vae,
    torch_dtype=torch.float16,
    variant="fp16",
    use_safetensors=True,
).to("cuda")
refiner.enable_xformers_memory_efficient_attention()

In [ ]:
#@title (Optional) Change sampler
from diffusers import EulerAncestralDiscreteScheduler, DPMSolverMultistepScheduler, UniPCMultistepScheduler

choose_sampler = "Euler A" #@param ["Euler A", "DPM++", "DPM++ Karras", "UniPC"]

config = base.scheduler.config
match choose_sampler:
    case "Euler A": sampler = EulerAncestralDiscreteScheduler.from_config(config)
    case "DPM++": sampler = DPMSolverMultistepScheduler.from_config(config)
    case "DPM++ Karras":
        sampler = DPMSolverMultistepScheduler.from_config(config)
        sampler.use_karras_sigmas = True
    case "UniPC":
        sampler = UniPCMultistepScheduler.from_config(config)
    case _: print("Unknown sampler")
base.scheduler = sampler
refiner.scheduler = sampler
print(f"Sampler '{choose_sampler}' chosen")

In [14]:
#@title #Render ui
#@markdown For now ignores everything but prompts, size, and steps.
from  StableDiffusionUi.ColabUI.DiffusionPipelineUI import DiffusionPipelineUI

ui = DiffusionPipelineUI()

#Default values
ui.height_field.value = 1024
ui.width_field.value = 1024
ui.steps_field.value = 40

ui.render()

IntSlider(value=20, description='Steps: ', min=1)

FloatSlider(value=7.0, description='CFG: ', max=20.0, min=1.0, step=0.5)

IntText(value=-1, description='seed: ')

In [ ]:
#@title <font color='red'>Run</font> this cell to generate images
high_noise_frac = 0.8

image = base(
    prompt=ui.positive_prompt.value,
    negative_prompt=ui.negative_prompt.value,
    num_inference_steps=ui.steps_field.value,
    denoising_end=high_noise_frac,
    output_type="latent",
    height=ui.height_field.value, width=ui.width_field.value,
).images

result = refiner(
    prompt=ui.positive_prompt.value,
    negative_prompt=ui.negative_prompt.value,
    num_inference_steps=ui.steps_field.value,
    denoising_start=high_noise_frac,
    image=image,
    height=ui.height_field.value, width=ui.width_field.value,
).images

result[0]